In [ ]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langchain.chat_models import init_chat_model
from pydantic import BaseModel

llm = init_chat_model("openai:gpt-5-mini")

In [ ]:
class State(TypedDict):
    dish: str
    ingredients: list[dict]
    recipe_steps: str
    plating_instructions: str

class Ingredient(BaseModel):
    name: str
    quantity: str
    unit: str

class IngredientsOutput(BaseModel):
    ingredients: list[Ingredient]

In [ ]:
def list_ingredients(state: State):
    structured_llm = llm.with_structured_output(IngredientsOutput)
    response = structured_llm.invoke(f"List 5-8 ingredients needed to make {state["dish"]}")
    return {
        "ingredients": response.ingredients
    }


def create_recipe(state: State):
    response = llm.invoke(f"Write a step by step cooking instructions for {state["dish"]}, using these ingredients {state["ingredients"]}")
    return {
        "recipe_steps": response.content
    }

def describe_plating(state: State):
    response = llm.invoke(f"Describe how to beautifully plate this dish {state["dish"]} based on this recipe {state["recipe_steps"]}")
    return {
        "plating_instructions": response.content
    }

In [ ]:
graph_builder = StateGraph(State)

graph_builder.add_node("list_ingredients", list_ingredients)
graph_builder.add_node("create_recipe", create_recipe)
graph_builder.add_node("describe_plating", describe_plating)

graph_builder.add_edge(START, "list_ingredients")
graph_builder.add_edge("list_ingredients", "create_recipe")
graph_builder.add_edge("create_recipe", "describe_plating")
graph_builder.add_edge("describe_plating", END)

graph = graph_builder.compile()

In [ ]:
graph.invoke({"dish": "hummus"})